In [10]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import init_chat_model
import os, json
from geoalchemy2 import Geometry

In [11]:
@tool(description="这是一个询问分类器，用来分类用户的问题")
def query_classifier(query: str):
    prompt = """你是一个分类器
    我们现在有武汉大学的地理信息postgresql数据库
    数据库有postgis扩展
    你负责将用户的问题分类为以下三类之一:
    1. 可处理的地理查询
    2. 不能用数据库查询解决或者超出范围的地理查询
    3. 闲聊
    请根据用户的问题进行分类，并只返回分类结果，不要添加任何多余的信息。
    """
    model = create_agent(model="gpt-4.1-mini", system_prompt=prompt)
    result = model.invoke({"messages":[{"role": "user", "content": query}]})
    return result["messages"][-1].content

In [12]:
db_url = os.getenv("DB_URL").replace("...", os.getenv("DB_NAME"))
database = SQLDatabase.from_uri(db_url)
sqltoolkit = SQLDatabaseToolkit(db=database,llm=init_chat_model("gpt-4.1-mini"), top_k=5)
with open("data/whu_schema.json", "r", encoding="utf-8") as f:
    schema = json.load(f)

In [13]:
@tool(description="这是一个地理信息数据库查询专家，用来将用户的问题转换为sql查询语句，并执行查询，最后将查询结果返回给用户")
def sql_agent(query: str) -> str:
    prompt = f"""你是一个地理信息数据库查询专家
    我们现在有武汉大学的地理信息postgresql数据库
    这是数据库的schema信息:{schema}
    数据库有postgis扩展
    你负责将用户的问题转换为sql查询语句，并执行查询
    最后将查询结果返回给用户
    注意事项:
    1. 如果用户的问题中没有涉及地理信息，请礼貌地拒绝回答
    2. 如果用户的问题中涉及地理信息，请务必使用postgis函数进行查询
    3. 请确保生成的sql语句是正确的，并且能够在数据库中执行
    4. 不要在sql语句中使用*，请明确指定需要查询的字段
    5. 返回结果时，只返回查询结果，不要添加任何多余的信息
    """
    model = create_agent(model="gpt-4.1-mini", system_prompt=prompt, tools=sqltoolkit.get_tools())
    result = model.invoke({"messages":[{"role": "user", "content": query}]})
    return result["messages"][-1].content

In [14]:
prompt = """
你是一个多智能体系统的主代理
你有以下工具可用:
1. query_classifier: 这是一个询问分类器，用来分类用户的问题
2. sql_agent: 这是一个地理信息数据库查询专家，用来将用户的问题转换为sql查询语句，并执行查询，最后将查询结果返回给用户
当你收到用户的问题时，首先使用query_classifier对问题进行分类
如果分类结果是Geo_Query_Can_Solve:，则使用sql_agent处理用户的问题，并将结果返回给用户
如果分类结果是Geo_Query_Cannot_Solve:，则礼貌地告诉用户你无法处理该问题
如果分类结果是Chat_Query:，则礼貌地与用户进行闲聊
"""
agent = create_agent(model="openai:o4-mini",
                     tools=[query_classifier, sql_agent],
                     system_prompt=prompt)

In [15]:
qury = "武汉大学有几个池塘？"
for event in agent.stream(
    {"messages": [{"role": "user", "content": qury}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

武汉大学有几个池塘？
================================== Ai Message ==================================
Tool Calls:
  query_classifier (call_WkrIYdVlDJerCr997f6Ae0pT)
 Call ID: call_WkrIYdVlDJerCr997f6Ae0pT
  Args:
    query: 武汉大学有几个池塘？
================================= Tool Message =================================
Name: query_classifier

1. 可处理的地理查询
================================== Ai Message ==================================
Tool Calls:
  sql_agent (call_2HX07UZ1OerKlZS8lXRFv8rb)
 Call ID: call_2HX07UZ1OerKlZS8lXRFv8rb
  Args:
    query: 武汉大学 有 几个 池塘
================================= Tool Message =================================
Name: sql_agent

您的数据库当前对SQL查询的字段名访问存在问题，导致包含字段名的查询均不能正常执行，建议检查数据库配置或字段名使用规范。
请问我可以帮您查询关于武汉大学周边其他类型的地理信息吗？
================================== Ai Message ==================================

抱歉，当前系统因数据库配置问题暂时无法检索“武汉大学”的池塘数量。您可以稍后再试，或告诉我是否需要查询其他关于武汉大学的地理信息？


In [8]:
qury = "'明天吃什么？', 这是一个地理查询问题吗？"
for event in agent.stream(
    {"messages": [{"role": "user", "content": qury}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

'明天吃什么？', 这是一个地理查询问题吗？
================================== Ai Message ==================================
Tool Calls:
  query_classifier (call_OpBZPyGpETl7eSEp5ofXIBkZ)
 Call ID: call_OpBZPyGpETl7eSEp5ofXIBkZ
  Args:
    query: 明天吃什么？
================================= Tool Message =================================
Name: query_classifier

Chat_Query
================================== Ai Message ==================================

“明天吃什么？” 这不是一个地理查询问题，而是一个日常生活类的聊天问题。如果你想讨论菜单或食谱，我可以给你一些建议！你想了解哪方面的内容？例如：  
1. 早餐、午餐还是晚餐的推荐  
2. 中式还是西式菜肴  
3. 快手便餐还是特色料理  
4. 特定食材的食谱  

请告诉我你的偏好~


In [9]:
qury = "'华中科技大学有几个食堂？', 这是一个地理查询问题吗？"
for event in agent.stream(
    {"messages": [{"role": "user", "content": qury}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

'华中科技大学有几个食堂？', 这是一个地理查询问题吗？
================================== Ai Message ==================================
Tool Calls:
  query_classifier (call_JOfyL8qTyhdbpv5FwQm6pDtl)
 Call ID: call_JOfyL8qTyhdbpv5FwQm6pDtl
  Args:
    query: '华中科技大学有几个食堂？', 这是一个地理查询问题吗？
================================= Tool Message =================================
Name: query_classifier

Geo_Query_Cannot_Solve
================================== Ai Message ==================================

抱歉，这个问题超出了我的处理范围，无法为您解答。
